# Prétraitement des textes et verctorisations

In [1]:
# modules
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [3]:
# Charger les donnees
data_spon = pd.read_csv("../data/corpus_spon.csv")
data_prep = pd.read_csv("../data/corpus_prep.csv")

In [4]:
data_spon_q = data_spon["question"]
data_prep_q = data_prep["question"]

# Test de l'algorithme clustering hiérarchique

In [5]:
from sklearn.datasets._samples_generator import make_blobs
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from itertools import cycle

In [6]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import FrenchStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer



# application de la fonction TF-IDF
tokenizer = RegexpTokenizer(r"\w+")
#stemmer = FrenchStemmer()
stopwords_fr = stopwords.words('french')
vectorizer = TfidfVectorizer(tokenizer=tokenizer.tokenize, stop_words=stopwords_fr, analyzer="word", ngram_range=(1, 3), min_df=5, max_df=0.9)
X = vectorizer.fit_transform(data_spon_q)


tf_idf = pd.DataFrame(data=X.toarray(), columns=vectorizer.get_feature_names_out())


# merger avec le dataframe original
final_df = tf_idf

print("{} rows".format(final_df.shape[0]))
final_df.T.nlargest(5, 0)

634 rows


/home/lena/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,624,625,626,627,628,629,630,631,632,633
pourquoi,0.799509,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.554854,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0
ça,0.600654,0.0,0.660388,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.456372,0.0,0.0,0.000000,0.0,0.000000,0.0
a,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.329342,0.0,0.000000,0.0,0.0,0.325646,0.0,0.000000,0.0
a fait,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0
accord,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.365736,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.208917,0.0


In [7]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.snowball import FrenchStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
from scipy.cluster import hierarchy as sch


In [8]:
# Construction de l'arbre de clustering
linkage_matrix = linkage(X.todense(), 'average')
threshold = 1.6 * np.median(linkage_matrix[:, 2])
labels = fcluster(linkage_matrix, threshold, criterion='distance')

# Récupération des échantillons représentatifs de chaque cluster
representative_samples = []
for i in np.unique(labels):
    cluster_indices = np.where(labels == i)[0]
    cluster_distances = pairwise_distances(X[cluster_indices], metric='euclidean')
    representative_sample_index = cluster_indices[np.argmin(np.mean(cluster_distances, axis=0))]
    representative_samples.append(representative_sample_index)

# Extraction des caractéristiques représentatives de chaque cluster
representative_features = []
for i in np.unique(labels):
    representative_sample = X[representative_samples[i-1]].toarray().squeeze()
    feature_indices = np.argsort(representative_sample)[::-1][:10]  # 选择前10个TF-IDF值最高的特征
    feature_names = vectorizer.get_feature_names_out()
    representative_features.append([feature_names[idx] for idx in feature_indices])

# Impression des caractéristiques représentatives de chaque cluster
for i, features in enumerate(representative_features):
    print("Cluster {}: {}".format(i+1, features))

# Visualisation de l'arbre de clustering
plt.figure(figsize=(10, 7))
dendrogram(linkage_matrix, truncate_mode='lastp', p=50, leaf_rotation=90., 
           leaf_font_size=8., labels=labels, no_labels=False)
plt.xlabel('Sample index')
plt.ylabel('Distance')
plt.show()


NameError: name 'linkage' is not defined